<a href="https://colab.research.google.com/github/mansueli/Supa-Migrate/blob/main/Amazon_RDS_to_Supabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Supabase](https://raw.githubusercontent.com/supabase/supabase/master/packages/common/assets/images/supabase-logo-wordmark--light.svg)



#Amazon RDS to Supabase migration guide

In [1]:
#@title Select the `Engine` of the RDS Database & install required resources. { display-mode: "form" }
engine = 'Postgres' #@param ["Postgres", "MySQL", "MS SQL","SQLite"]

if engine=='Postgres':
  !sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
  !wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
  !sudo apt-get update &>log
  !sudo apt-get -y install postgresql &>log
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/migrate_postgres_project.sh &>log
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/after.sql
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/before.sql
  print("Installed Postgres")
else:
  !wget https://github.com/dimitri/pgloader/archive/refs/tags/v3.6.9.zip &>log
  !apt-get install sbcl unzip libsqlite3-dev make curl gawk freetds-dev libzip-dev &>log
  !unzip v3.6.9.zip -d pgloader &>log
  !make -C ./pgloader/pgloader-3.6.9  &>log
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --help
  print("Pgloader installed to migrated "+engine+ "to Supabase")

OK
--2023-12-12 21:27:12--  https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/after.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79 [text/plain]
Saving to: ‘after.sql’

after.sql           100%[===================>]      79  --.-KB/s    in 0s      

2023-12-12 21:27:12 (3.03 MB/s) - ‘after.sql’ saved [79/79]

--2023-12-12 21:27:12--  https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/before.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74 [text/plain]
Saving to: ‘before.sql’

before.sql          1

## Set the environment Variables:

In [2]:
#Source DB variables:
%env HOST=duplicate-of-workweek-prod-for-testing.c4ffsv6b9ash.eu-west-1.rds.amazonaws.com
%env USER=idevadmin
%env SOURCE_DB=postgres
%env PASSWORD=d9dGJhZgwB3kykQpqXGXeFYW9aU
%env PORT=5432
#Supabase variables:
%env SUPABASE_URL=https://ytlqeucmuaongxxdneal.supabase.co
%env SUPABASE_PASSWORD=d9dGJhZgwB3kykQpqXGXeFYW9aU
%env SUPABASE_PORT=5432

env: HOST=duplicate-of-workweek-prod-for-testing.c4ffsv6b9ash.eu-west-1.rds.amazonaws.com
env: USER=idevadmin
env: SOURCE_DB=postgres
env: PASSWORD=d9dGJhZgwB3kykQpqXGXeFYW9aU
env: PORT=5432
env: SUPABASE_URL=https://ytlqeucmuaongxxdneal.supabase.co
env: SUPABASE_PASSWORD=d9dGJhZgwB3kykQpqXGXeFYW9aU
env: SUPABASE_PORT=5432


In [4]:
#@title #Running the Migration: { display-mode: "form" }
import os

supabase_url = os.environ['SUPABASE_URL']

if engine=='MySQL':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --before before.sql --after after.sql 'mysql://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB' 'pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:$SUPABASE_PORT/postgres' #&>log
if engine=='MS SQL':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --before before.sql --after after.sql 'mssql://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB' 'pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:$SUPABASE_PORT/postgres' #&>log
if engine=='SQLite':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --before before.sql --after after.sql 'sqllite://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB' 'pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:$SUPABASE_PORT/postgres' #&>log
if engine=='Postgres':
  sender_db = os.environ['SOURCE_DB']
  sender_user = os.environ['USER']
  sender_host = os.environ['HOST']
  sender_pgpass = os.environ['PASSWORD']
  filedata = ''
  with open('migrate_postgres_project.sh', 'r') as file :
    filedata = file.read()
    filedata = filedata.replace('POSTGRES_ORIGIN_USERNAME', sender_user)
    filedata = filedata.replace('POSTGRES_ORIGIN_PASSWORD', sender_pgpass.replace("PGPASSWORD=",""))
    filedata = filedata.replace('POSTGRES_ORIGIN_DATABASE', sender_db)
    filedata = filedata.replace('POSTGRES_ORIGIN_HOST', sender_host)
    filedata = filedata.replace('SUPABASE_HOST', os.environ["SUPABASE_URL"].replace('https://','db.'))
    filedata = filedata.replace('SUPABASE_PASSWORD', os.environ['SUPABASE_PASSWORD'])

  with open('migrate_postgres_project.sh', 'w') as file:
    file.write(filedata)

  !bash ./migrate_postgres_project.sh #&>log
print("Migration completed")

pg_dump: error: connection to server at "duplicate-of-workweek-prod-for-testing.c4ffsv6b9ash.eu-west-1.rds.amazonaws.com" (54.78.105.231), port 5432 failed: FATAL:  password authentication failed for user "idevadmin"
connection to server at "duplicate-of-workweek-prod-for-testing.c4ffsv6b9ash.eu-west-1.rds.amazonaws.com" (54.78.105.231), port 5432 failed: FATAL:  password authentication failed for user "idevadmin"
pg_dump: error: options -c/--clean and -a/--data-only cannot be used together
psql: error: could not translate host name "db.ytlqeucmuaongxxdneal.supabase.co" to address: Name or service not known
psql: error: could not translate host name "db.ytlqeucmuaongxxdneal.supabase.co" to address: Name or service not known
Migration completed
